# Antimatter rocket

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lookbusy1344/Relativity/blob/main/Python/Antimatter%20rocket.ipynb)

This repository is primarily focused on the purely kinematic equations of Special Relativity. However, it also includes some basic functions for an antimatter rocket, in both Python and Typescript. These are used in the interactive web calculator:

https://lookbusy1344.github.io/Relativity/

The Python propulsion functions are in https://github.com/lookbusy1344/Relativity/blob/main/Python/propulsion.py

## Two kinds of antimatter rocket

There are two basic kinds of antimatter rocket:

1. **Photonic rocket**: using the energy from matter-antimatter annihilation to produce photons (gamma rays) which are emitted out the back of the rocket to produce thrust. This is the simplest kind of antimatter rocket, but is unrealistic because it is very difficult to collimate the gamma rays into a beam.

2. **Charged-pion rocket**: using charged pions produced in proton-antiproton annihilation to produce thrust. NASA experimental studies show that approximately **40% of the annihilation energy** appears as charged pion kinetic energy that can be magnetically collimated for thrust. The remaining ~60% goes to neutral pions (which decay to gamma rays), pion rest mass energy, and other particles that cannot be efficiently directed.

## We use the charged-pion rocket equations here

This technology is beyond our current capabilities, but is scientifically plausible. We assume various levels of nozzle efficiency to account for engineering challenges in collimating the charged pions. A nozzle efficiency of 85% with 40% energy in charged pions gives **~34% total system efficiency**, which is probably near the maximum achievable.

**References:**
- NASA antimatter propulsion studies: https://ntrs.nasa.gov/api/citations/20200001904/downloads/20200001904.pdf
- NASA contractor report: https://ntrs.nasa.gov/api/citations/19890018329/downloads/19890018329.pdf
- Experimental data: https://link.springer.com/article/10.1140/epja/s10050-024-01428-x

In [5]:
# Download propulsion.py from your GitHub repo if not present. Important for colab.
import os

if not os.path.exists("relativity_lib.py"):
    !wget -q https://raw.githubusercontent.com/lookbusy1344/Relativity/main/Python/relativity_lib.py

if not os.path.exists("propulsion.py"):
    !wget -q https://raw.githubusercontent.com/lookbusy1344/Relativity/main/Python/propulsion.py

import mpmath as mp
import relativity_lib as rl
import propulsion as prop
from prettytable import PrettyTable, TableStyle
from IPython.display import Markdown

# Configure relativity_lib for precision calculations
rl.configure(100)

fuel_mass = mp.mpf(1000)  # kg, half is matter, half is antimatter
dry_mass = mp.mpf(500)  # kg, the spacecraft dry mass

t = prop.pion_rocket_accel_time(fuel_mass, dry_mass, nozzle_efficiency=0.85)
years = t / rl.seconds_per_year
print(
    f"Dry mass {float(dry_mass):.0f}, fuel mass {float(fuel_mass):.0f} means {float(years):.2f} years at 1g "
    f"(85% nozzle efficiency, ~34% total system efficiency)"
)
print()

# Calculate propellant fraction for 3.52 years of 1g acceleration at 85% nozzle efficiency
years = rl.ensure("3.52")
secs = rl.seconds_per_year * years
f = prop.pion_rocket_fuel_fraction(secs, None, 0.85)
print(
    f"Propellant mass fraction: {float(f) * 100:.4f}% for {float(years)} years of 1g "
    f"(85% nozzle efficiency, ~34% total system efficiency)"
)

table = PrettyTable(
    [
        "Nozzle efficiency",
        "Proper time (years)",
        "Propellant fraction (%)",
    ]
)
table.set_style(TableStyle.MARKDOWN)
table.align = "r"  # right align everything

perc = mp.mpf("100")

# Define efficiencies and time periods
nozzle_efficiencies = [0.70, 0.75, 0.80, 0.85]
proper_times_years = [0.01, 0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 48]

# Populate table with combinations
for eff in nozzle_efficiencies:
    for years in proper_times_years:
        secs = rl.seconds_per_year * rl.ensure(str(years))
        f = prop.pion_rocket_fuel_fraction(secs, None, eff)
        frac = rl.format_mpf_significant(f * perc, 2, "9")

        table.add_row([f"{eff:.0%}", years, frac])

print()
print(
    "Here we show propellant fractions (how much of the ship's mass needs to be fuel) for various nozzle efficiencies and proper times at 1g acceleration"
)
print("Nozzle efficiencies range from 70% to 85%, and various durations")
markdown_table = table.get_string()
display(Markdown(markdown_table));

Dry mass 500, fuel mass 1000 means 0.34 years at 1g (85% nozzle efficiency, ~34% total system efficiency)

Propellant mass fraction: 99.9988% for 3.52 years of 1g (85% nozzle efficiency, ~34% total system efficiency)

Here we show propellant fractions (how much of the ship's mass needs to be fuel) for various nozzle efficiencies and proper times at 1g acceleration
Nozzle efficiencies range from 70% to 85%, and various durations


| Nozzle efficiency | Proper time (years) |                                                              Propellant fraction (%) |
|-----------------: |-------------------: |------------------------------------------------------------------------------------: |
|               70% |                0.01 |                                                                                 3.84 |
|               70% |                 0.1 |                                                                                32.44 |
|               70% |                   1 |                                                                                98.02 |
|               70% |                   2 |                                                                               99.960 |
|               70% |                   3 |                                                                             99.99922 |
|               70% |                   4 |                                                                            99.999984 |
|               70% |                   5 |                                                                          99.99999969 |
|               70% |                   6 |                                                                        99.9999999939 |
|               70% |                   7 |                                                                       99.99999999988 |
|               70% |                   8 |                                                                     99.9999999999976 |
|               70% |                   9 |                                                                   99.999999999999953 |
|               70% |                  10 |                                                                 99.99999999999999907 |
|               70% |                  15 |                                                         99.9999999999999999999999971 |
|               70% |                  20 |                                                99.9999999999999999999999999999999914 |
|               70% |                  25 |                                        99.999999999999999999999999999999999999999973 |
|               70% |                  48 | 99.999999999999999999999999999999999999999999999999999999999999999999999999999999982 |
|               75% |                0.01 |                                                                                 3.59 |
|               75% |                 0.1 |                                                                                30.65 |
|               75% |                   1 |                                                                                97.42 |
|               75% |                   2 |                                                                               99.933 |
|               75% |                   3 |                                                                              99.9982 |
|               75% |                   4 |                                                                            99.999956 |
|               75% |                   5 |                                                                           99.9999988 |
|               75% |                   6 |                                                                         99.999999971 |
|               75% |                   7 |                                                                       99.99999999925 |
|               75% |                   8 |                                                                      99.999999999980 |
|               75% |                   9 |                                                                    99.99999999999950 |
|               75% |                  10 |                                                                   99.999999999999987 |
|               75% |                  15 |                                                           99.99999999999999999999985 |
|               75% |                  20 |                                                   99.9999999999999999999999999999983 |
|               75% |                  25 |                                           99.999999999999999999999999999999999999981 |
|               75% |                  48 |      99.9999999999999999999999999999999999999999999999999999999999999999999999999951 |
|               80% |                0.01 |                                                                                 3.37 |
|               80% |                 0.1 |                                                                                29.04 |
|               80% |                   1 |                                                                                96.76 |
|               80% |                   2 |                                                                                99.89 |
|               80% |                   3 |                                                                              99.9966 |
|               80% |                   4 |                                                                             99.99989 |
|               80% |                   5 |                                                                           99.9999964 |
|               80% |                   6 |                                                                          99.99999988 |
|               80% |                   7 |                                                                        99.9999999963 |
|               80% |                   8 |                                                                       99.99999999988 |
|               80% |                   9 |                                                                     99.9999999999961 |
|               80% |                  10 |                                                                    99.99999999999987 |
|               80% |                  15 |                                                            99.9999999999999999999955 |
|               80% |                  20 |                                                     99.99999999999999999999999999984 |
|               80% |                  25 |                                             99.9999999999999999999999999999999999945 |
|               80% |                  48 |           99.99999999999999999999999999999999999999999999999999999999999999999999971 |
|               85% |                0.01 |                                                                                 3.17 |
|               85% |                 0.1 |                                                                                27.60 |
|               85% |                   1 |                                                                                96.04 |
|               85% |                   2 |                                                                                99.84 |
|               85% |                   3 |                                                                              99.9938 |
|               85% |                   4 |                                                                             99.99975 |
|               85% |                   5 |                                                                           99.9999903 |
|               85% |                   6 |                                                                          99.99999961 |
|               85% |                   7 |                                                                         99.999999984 |
|               85% |                   8 |                                                                       99.99999999940 |
|               85% |                   9 |                                                                      99.999999999976 |
|               85% |                  10 |                                                                    99.99999999999906 |
|               85% |                  15 |                                                             99.999999999999999999909 |
|               85% |                  20 |                                                      99.9999999999999999999999999911 |
|               85% |                  25 |                                               99.99999999999999999999999999999999914 |
|               85% |                  48 |               99.9999999999999999999999999999999999999999999999999999999999999999953 |

## Crossing the observable universe

48 years of proper time at 1g is sufficient to cross the observable universe, flipping half way to return to the original frame, see [this notebook for details](https://github.com/lookbusy1344/Relativity/blob/main/Python/Universe.ipynb).

However such a journey would require an inconceivable propellant fraction. Even without slowing down to the original frame, the propellant fraction is still enormous. Lets work out the required fuel mass.

In [6]:
dry_mass = mp.mpf("500")  # kg
eff = 0.85  # nozzle efficiency
secs = rl.seconds_per_year * mp.mpf(
    "47.67"
)  # 47.67 proper years for 47 billion light years at 1g
galaxy = mp.mpf("3e42")  # kg, mass of Milky Way

fraction = prop.pion_rocket_fuel_fraction(secs, None, eff)
fuel_mass = dry_mass * (fraction / (1 - fraction))
print()
print(f"To cross the observable universe at 1g with 85% nozzle efficiency")
print(f"Mass fraction for fuel {rl.format_mpf_significant(fraction, 2, '9')}")
# print(f"The fuel mass required is {rl.format_mpf(fuel_mass,0)} kg")
print()
print(f"Ship dry mass of {rl.format_mpf(dry_mass, 0)} kg")
print(f"The fuel mass required is {rl.format_mpf_sci(fuel_mass, 4)} kg")
print(f"Mass of Milky Way is {rl.format_mpf_sci(galaxy)} kg")
print()
print(f"This is {rl.format_mpf(fuel_mass / galaxy, 1)} times the mass of the Milky Way")


To cross the observable universe at 1g with 85% nozzle efficiency
Mass fraction for fuel 0.99999999999999999999999999999999999999999999999999999999999999999986

Ship dry mass of 500 kg
The fuel mass required is 3.7e+69 kg
Mass of Milky Way is 3.0e+42 kg

This is 1,233,464,005,645,591,707,482,156,399.4 times the mass of the Milky Way


## Conclusion

To cross the observable universe in a human lifetime, even with an antimatter rocket, you would need around **1.2 octillion times the mass of the Milky Way** as fuel, for a small 500 kg ship. This is clearly ridiculous, and shows the practical limits of intergalactic travel even with the most advanced propulsion technology we can currently conceive of.

More relativity notebooks are [available here](https://github.com/lookbusy1344/Relativity/blob/main/Python/README.md)

You can read about antimatter rockets in more detail on [Wikipedia](https://en.wikipedia.org/wiki/Antimatter_rocket).